In [1]:
import time
import datetime as dt
import pandas as pd
import snowflake.connector
from pypardot.client import PardotAPI, PardotAPIError
import snowflake.connector
import requests


In [2]:
PARDOT_EMAIL="segment_integrations@discoveryed.com"
PARDOT_USER_KEY="f5a1dc61d079e35d9a2066a4a8498c32"
PARDOT_SF_CONSUMER_KEY="3MVG9IHf89I1t8hpom1l0QzbTANHH.MOASIZ3yPPhu6hFI_uipXiYi7ku85yMWg_2gXxu5bzAyWCevOHp4jrf"
PARDOT_SF_CONSUMER_SECRET="BC6C2AA5D41DC0EBBBB3376CD329E92E66BD8D85DF85F9FAB9F22E86D857A29E"
PARDOT_SF_REFRESH_TOKEN="5Aep8618yVsldz6rZPMv4ouelrGoRNAfdaLSjo3.ILw4jdNGemJVI_MNYFoNkN9g39GUXqUEgzw3ylEVMjxAU3X"
PARDOT_BUSINESS_UNIT_ID="0Uv4P000000TNB0SAO"
PARDOT_API_VERSION=4
PARDOT_MAX_RESULT_COUNT = 200
SNOWFLAKE_ACCOUNT_IDENTIFIER="UK29315.us-east-1"
SNOWFLAKE_USER="PardotEtlUser@discoveryed.com"
SNOWFLAKE_PASS="p@rd0tEtlUser"
SF_SCHEMA = "PROD_DATA_VAULT"

In [85]:
def get_client_pardot() -> PardotAPI:
    try:
        p = PardotAPI(
            email=PARDOT_EMAIL,
            user_key=PARDOT_USER_KEY,
            sf_consumer_key=PARDOT_SF_CONSUMER_KEY,
            sf_consumer_secret=PARDOT_SF_CONSUMER_SECRET,
            sf_refresh_token=PARDOT_SF_REFRESH_TOKEN,
            business_unit_id=PARDOT_BUSINESS_UNIT_ID,
            version=PARDOT_API_VERSION,
        )

        if not p.authenticate():
            p.campaigns.query(limit=1)
            # ^ Must call query to authenticate if authenticatation is unsuccessful

    except PardotAPIError as err:
        raise

    assert (
        p.sftoken != "dummy"
    ), "Pardot authentication still unsuccessful after querying"

    return p

def get_client_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASS,
        account=SNOWFLAKE_ACCOUNT_IDENTIFIER,
        # authenticator="externalbrowser",
        warehouse="GENERAL_COMPUTE_WH",
        database="PROD_DATA_VAULT",
        schema="STAGE",
    )

def query_sf(query_str):
    ctx = get_client_snowflake()
    cs = ctx.cursor()
    cs.execute(query_str)
    data =  cs.fetchall()  
    columns = [f'{col[0]}' for col in cs.description]
    return pd.DataFrame(data,columns=columns)

In [44]:
p = get_client_pardot()

In [102]:
tables = [
'Campaign',
'Form',
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account',
'TagObject',
"Prospect",
"VisitorActivity",
"Visitor",
"ListMembership",
"ProspectAccount"
]

tag_object_types = ["Email","Campaign","List"]

DICT_CONVERSION_PURAL = {
    "Account": "accounts",
    "EmailClick": "emailclicks",
    "Email": "emails",
    "VisitorActivity": "visitoractivities",
    "Campaign": "campaigns",
    "List": "lists",
    "Prospect": "prospects",
    "TagObject": "tagobjects",
    "Tag": "tags",
    "Opportunity": "opportunities",
    "ProspectAccounts": "prospectaccounts",
    "Form": "forms",
    "ListMembership": "listmemberships",
    "Visitor": "visitors",
    "ProspectAccount": "prospectaccounts",
}


In [55]:
def helper_to_camelCase(s: str) -> str:
    "Return the same string with the first letter lowercase"
    return str(s[0].lower() + s[1:])

In [104]:
expected_results = {}
for data_type in tables:
    try:
        print(data_type)
        data_client = getattr(p, DICT_CONVERSION_PURAL[data_type])
        if not hasattr(data_client, "query"):
            # If the data_client doesn't have a query method, try the "read" method
            data = data_client.read()[helper_to_camelCase(data_type)]
            if isinstance(data, dict):
                # Assume only one record
                expected_results[data_type]=1
            elif isinstance(data, list):
                expected_results[data_type]=len(data)
        else:
            if data_type=="TagObject":
                counter = 0
                for tag_type in tag_object_types:
                    time.sleep(2)
                    data_raw = data_client.query(limit=1,type=tag_type)
                    counter = counter + data_raw["total_results"]
                expected_results[data_type] = counter
            else:
                data_raw = data_client.query(limit=1)
                expected_results[data_type]=data_raw["total_results"]
    except:
        print(f"{data_type} failed")
        raise



Campaign
Form
Tag
Opportunity
EmailClick
List
Account
TagObject
Prospect
VisitorActivity
Visitor
ListMembership
ProspectAccount


In [100]:
tables = ['Campaign',
'Form',
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account',
'TagObject',
"Email",
"Prospect",
"VisitorActivity",
"Visitor",
"ListMembership",
"ProspectAccount"]

queries = []
for table in tables:
    if table == "TagObject":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct tag_object_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct tag_object_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    elif table == "VisitorActivity":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct visitor_activity_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct visitor_activity_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    else:
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct {table}_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct {table}_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    queries.append(sql_template)

query_join = ' union all '.join(queries)

email_query = """select 
        count(distinct a.list_email_id) as expected
        from PARDOT_VISITORACTIVITY a
        where a.email_id is not null and a.list_email_id is not null"""

sf_data = query_sf(query_join)

try:
    ctx = get_client_snowflake()
    cs = ctx.cursor()
    cs.execute(email_query)
    email_expected =  cs.fetchone()[0]
finally:
    cs.close()
ctx.close()

# Add expected count for email object
sf_data.loc[sf_data["DATA_OBJECT"]=="Email","EXPECTED"] = email_expected

In [105]:
# Add expected count for all the other data objects
for d_obj, exected in expected_results.items():
    sf_data.loc[sf_data["DATA_OBJECT"]==d_obj,"EXPECTED"] = exected

# Determine discrepancy between expected and whats in stage
sf_data["DIFFERENCE"] = sf_data["EXPECTED"]-sf_data["COUNT_STAGE_DISTINCT"]

# Issues with # of records compared to whats expected to come out of Pardot
# It's almost never going to be perfect, but just check that it's relatively close
sf_data.loc[sf_data["EXPECTED"]<sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "EXPECTED LOWER"
sf_data.loc[sf_data["EXPECTED"]>sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "DATA MISSING"
sf_data.loc[sf_data["EXPECTED"]==sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "CORRECT"
# Issues with duplicates in marketing schema
sf_data.loc[sf_data["COUNT_MARKET_DISTINCT"]<sf_data["COUNT_MARKET_TOTAL"],"MARKET_ISSUE"] = "DUPLICATES IN MARKET"

sf_data

# DATA OBJECT - which dataset this refers to
# COUNT_MARKET_DISTINCT - how many distinct IDs are in the marketing schema table
# COUNT_MARKET_TOTAL - how many total IDs are in the marketing schema table (this is used to check for duplicates against COUNT_MARKET_DISTINCT)
# COUNT_STAGE_DISTINCT - how many distinct IDs are in the stage schema table (this is compared to COUNT_MARKET_DISTINCT to determine if all records from stage have been synced to marketing)
# COUNT_STAGE_TOTAL - how many total IDs are in the stage schema table
# EXPECTED - This is how many records for each dataset currently exist in pardot. We expected COUNT_STAGE_DISTINCT to be very close to this number
# DIFFERENCE - Difference between EXPECTED and COUNT_STAGE_DISTINCT. Negative numbers mean data is in the warehouse that potentially shouldn't be. Positive means we are missing data. Just need to check it's close
# EXPECT_ISSUE - Flags if data is missing or if extra data exists
# MARKET_ISSUE - Checks for duplicate IDs in the marketing schema


,DATA_OBJECT,COUNT_MARKET_DISTINCT,COUNT_MARKET_TOTAL,COUNT_STAGE_DISTINCT,COUNT_STAGE_TOTAL,EXPECTED,DIFFERENCE,EXPECT_ISSUE,MARKET_ISSUE
0,Campaign,552,552,552,1655,552.0,0.0,CORRECT,NaN
1,Form,14,14,14,27,14.0,0.0,CORRECT,NaN
2,Tag,174,174,174,178,174.0,0.0,CORRECT,NaN
3,Opportunity,8499,8499,8503,12014,8503.0,0.0,CORRECT,NaN
4,EmailClick,1142960,1142960,1143599,1144830,1143599.0,0.0,CORRECT,NaN
5,List,1800,1800,1803,1809,1801.0,-2.0,EXPECTED LOWER,NaN
6,Account,1,1,1,3,1.0,0.0,CORRECT,NaN
7,TagObject,3485,3485,3491,3510,3505.0,14.0,DATA MISSING,NaN
8,Email,0,0,7886,7886,7886.0,0.0,CORRECT,NaN
9,Prospect,4714969,4714969,4715487,4716028,4716658.0,1171.0,DATA MISSING,NaN


In [106]:
# CHECK ON THE STATUS OF A BULK EXPORT (USE THE ID LISTED IN THE CLOUDWATCH LOGS)
headers = {
    "content-type": "application/json",
    "Authorization": f"Bearer {p.sftoken}",
    "Pardot-Business-Unit-Id": PARDOT_BUSINESS_UNIT_ID,
}
PARDOT_API_VERSION = 4
ID = 4522 #update this
PARDOT_URL_API = f"https://pi.pardot.com/api/export/version/{PARDOT_API_VERSION}/do/read/id/{ID}"

response = requests.get(
    PARDOT_URL_API,
            headers=headers,
            params=(("format", "json"),),
        )

response.json()["export"]["state"]
 

'Waiting'

,MAX(UPDATED_AT),MIN(UPDATED_AT)
0,2020-12-31 22:46:11-08:00,2020-03-24 11:16:50-07:00


In [52]:
# Calendar Table => prod_looker.dbt.calendar

ctx = get_client_snowflake()
cs = ctx.cursor()
name_table_snowflake = "PARDOT_OPPORTUNITY"
SF_SCHEMA = "PROD_DATA_VAULT"
cs.execute("select * from prod_looker.dbt.calendar limit 1")
data =  cs.fetchall()  
columns = [f'{col[0]}' for col in cs.description]

pd.DataFrame(data,columns=columns)


,DATE,DATEKEY,DAY_OF_MONTH,DAY_NAME,DAY_OF_WEEK,DAY_OF_YEAR,WEEK_OF_YEAR,MONTH_OF_YEAR,MONTH_NAME,QUARTER_OF_YEAR,...,SCHOOL_YEAR,SCHOOL_YEAR_START,SCHOOL_YEAR_END,PREV_SCHOOL_YEAR_START,PREV_SCHOOL_YEAR_END,ADJUSTED_SCHOOL_YEAR,ADJUSTED_WEEK_OF_SCHOOL_YEAR,ADJUSTED_DAY_OF_SCHOOL_YEAR,ADJUSTED_SCHOOL_YEAR_START,WEEK_OF_SCHOOL_YEAR_LABEL
0,2005-10-10,20051010,10,Mon,1,283,41,10,Oct,4,...,2005-2006,2005-07-01,2006-06-30,2004-07-01,2005-06-30,2005-2006,15,99,2005-07-04,W15 - Oct
